<a href="https://colab.research.google.com/github/partabparmar/Certifacates_in_Python/blob/main/Gemma_RAG_With_MongoDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step-by-Step Tutorial to Build a RAG using Google Gemma and MongoDB Atlas
In this step-by-step tutorial, we will use a simple Retrieval Augmented Generation (RAG) with Google Gemma to help it answer travel-related queries.

Before we start, let’s look into what RAG is.

**Retrieval Augmented Generation**, aka RAG, is often described as an “open-book” approach to answering domain-specific questions.

Typically, Large Language Models (LLMs) are trained on vast amounts of data, but sometimes they need updates to provide accurate information for specific topics.

There are two main methods:


1. Fine-tuning the model with specific data or giving it extra context with user queries.

2. Using Retrieval Augmented Generation (RAG): This is like an open-book test for the model, where it can find relevant answers. This approach is often preferred due to lower computational costs compared to fine-tuning.

For more details, you can explore RAG [here](https://arxiv.org/abs/2005.11401).

Now, let’s talk about enhancing Google Gemma for travel-related questions.

Below is our architecture.

![RAG](https://drive.google.com/uc?id=1Y6IT6wKrDwjOIX-3IZlq5LVlHRcEHebC)

## Step 0: Install Dependencies

In [ ]:
!pip install datasets pymongo sentence_transformers gradio

## Step 1: Building our Documents Database
![Dataset loading](https://drive.google.com/uc?id=1fC7fCFRxfgzC1sfemaJEK5fGlk3yqqpY)

We assembled our Documents database using information from Wikivoyage, a freely accessible online travel guide authored by volunteers.

>*Note: While this initial iteration suffices for prototyping purposes, future iterations could benefit from additional data and chunking techniques for enhanced production readiness.*

However, for this tutorial, we have already processed and parsed the data for you and will be directly using it from HuggingFace.

We will be using the `ashmib/wikivoyage-eu-city-embeddings` dataset here.
This dataset comprises abstracts from `Wikivoyage` for 160 European cities along with their corresponding country `names`, `coordinates`, and `populations`. The embeddings are derived from the `GTE-Large model`, incorporating data from the city, country, population, and abstract columns.

We load it from HuggingFace and transform it into a `pandas` dataframe.

Link to the dataset: https://huggingface.co/datasets/ashmib/wikivoyage-eu-city-embeddings




In [ ]:
from datasets import load_dataset
dataset = load_dataset("ashmib/wikivoyage-eu-city-embeddings", download_mode="force_redownload") ## downloading it from HuggingFace
dataset.set_format(type='pandas') ## converting it into pandas
df = dataset["train"][:]
df.head()

## Step 2: Embedding Model
![Embedding.png](https://drive.google.com/uc?id=1yMntmFJdugi1ib-JNOoqlo_WbPgWvDy3)

The next step is to generate the embeddings that will be used to embed both the documents and the queries.


> Embeddings are vector representations of textual data, aiding in semantic understanding and similarity comparisons.


The embeddings enhance the efficiency and accuracy of information retrieval systems by encoding semantic relationships between documents and queries.

In this case, we utilize the `gte-large` embedding model from the `SentenceTransformer` library to generate embeddings for documents and queries.

> *Note: In practice, you should also compute the embeddings for your documents. However, for this tutorial, we have you covered. We will only use the embeddings function to compute the embeddings for the user query.*



In [ ]:
from sentence_transformers import SentenceTransformer

def get_embedding(text: str) -> list[float]:

    embedding_model = SentenceTransformer("thenlper/gte-large")
    if not text.strip():
        print("Attempted to get embedding for empty text.")
        return []

    embedding = embedding_model.encode(text)

    return embedding.tolist()

## Step 3: Vector Database Setup & Data Ingestion

Here, we use MongoDB as the operational and vector database.

![Vector DB setup](https://drive.google.com/uc?id=1Z82OKKV72GvEDuhjRnFkMkGiuZ73BD3k)

1. We create an account on [MongoDB Atlas](https://www.mongodb.com/cloud/atlas/register) and note the username, password and connection string (Mongo URI).
2. Add the username and password as secrets to the Colab or environment variables
3. We utilize the `pymongo` library to establish database connectivity
4. Then we migrate our tabular data into the collections we just created using

    ```
documents = df.to_dict('records')
collection.insert_many(documents)
    ```



In [ ]:
import pymongo
import os
from google.colab import userdata

def get_mongo_url():
    username = userdata.get("MONGO_USERNAME")
    password = userdata.get("MONGO_PW")
    # uri_string = "@cluster0.62unmco.mongodb.net/"
    uri_string = userdata.get("MONGO_CONN_STR")
    mongo_url = f"mongodb+srv://{username}:{password}{uri_string}"
    return mongo_url


def get_mongo_client(mongo_url):
    """Establish connection to the MongoDB."""
    if not mongo_url:
        print("MONGO_URI not set in environment variables")
    try:
        client = pymongo.MongoClient(mongo_url)
        print("Connection to MongoDB successful")
        return client
    except pymongo.errors.ConnectionFailure as e:
        print(f"Connection failed: {e}")
        return None

In [ ]:
## Data ingestion in Mongodb

mongo_url = get_mongo_url()
if not mongo_url:
    print("MONGO_creds not set in environment variables")

# establishes database connection
mongo_client = get_mongo_client(mongo_url)
# creates database
db = mongo_client["wikivoyage_cities"]
# creates collection
collection = db["wikivoyage_collection_2"]

# Delete any existing records in the collection
collection.delete_many({})

# data ingestion into mongoDB
documents = df.to_dict('records')
collection.insert_many(documents)
print("Data ingestion into MongoDB completed")

## Step 4: Retrieval
![Retrieval](https://drive.google.com/uc?id=1Qnk-v5ZGFo1DN3Ye6iylcL4hOL9ATHjR)

In this step, we retrieve the most similar documents to the query from the vector database.

In the `query_results` function
1. `[Line 2-3]`: We connect to the Mongo Client
2. `[Line 5]`: We retrieve the embedding for the input query using the `get_embedding` function
3. `[Line 6-16]`: The MongoDB aggregation framework is then used to perform a `vector search` on the collection named `“EU_cities_collection.”` The vector search is performed using a specified index (`“vector_index”`) and the path to the embedding field, using the query_embedding as the vector for the search. It specifies that up to 150 candidates should be retrieved but limits the final results to the top 5 candidates.
4. In the `get_search_result` function:

    We begin by calling the `query_results` function with the provided `query` and `mongo_url`. This returns the `5 top most similar documents` from our database, i.e., the context that is then augmented with the prompt later.

In [ ]:
def query_results(query, mongo_url):
    mongo_client = get_mongo_client(mongo_url)
    db = mongo_client["wikivoyage_cities"]

    query_embedding = get_embedding(query)
    results = db.wikivoyage_collection_2.aggregate([
        {
            "$vectorSearch": {
                "index": "vector_index",
                "path": "embedding",
                "queryVector": query_embedding,
                "numCandidates": 150,
                "limit": 5
            }
        }
    ])
    return results

def get_search_result(query, mongo_url):
    get_knowledge = query_results(query, mongo_url)
    print(get_knowledge)

    search_result = ""
    for result in get_knowledge:
        search_result += f"City: {result.get('city', 'N/A')}, Abstract: {result.get('combined', 'N/A')}\n"

    return search_result

## Step 5: Augmentation and Generation
![Augmentation & Generation](https://drive.google.com/uc?id=11-Hn2WbD8_KqsQpxny9ovH1IZa78r7PW)

0. From the above steps, we have our context (derived from the `get_search_result` function with the user query).
1. We combine it with the user prompt to create an augmented prompt.
2. This augmented prompt serves as an input for our chosen LLm to generate a context specific result.
3. In this instance, we utilize the `google/gemma-2b-it` model from the `HuggingFace InferenceClient` for `text generation`.
    Link to the model: https://huggingface.co/google/gemma-2b

In [ ]:
from huggingface_hub import InferenceClient

HF_token = userdata.get("HF_TOKEN")

def generate_text(query, model_name: str | None = "google/gemma-2b-it"):
    if model_name is None:
        model_name = "google/gemma-2b-it"

    # establish mongo connection
    mongo_url = get_mongo_url()

    # get the top 5 most similar documents from the Vector data base
    source_information = get_search_result(query, mongo_url)

    # augment the query with the context
    combined_information = (
        f"Query: {query}\nContinue to answer the query by using the Search Results:\n{source_information}."
    )

    # use the HF inference client to generate the text
    client = InferenceClient(model_name, token=HF_token)
    stream = client.text_generation(prompt=combined_information, details=True, stream=True, max_new_tokens=2048,
                                    return_full_text=False)
   # formatting the output
    output = ""

    for response in stream:
        output += response.token.text

    if "<eos>" in output:
        output = output.split("<eos>")[0]
    return output

In [ ]:
query = "I am planning a vacation to Spain. Can you suggest a one-week itinerary including must-visit places and local cuisines to try?"
generate_text(query)

## Step 6: Gradio Interface
[Gradio](https://www.gradio.app/docs) is an open-source Python package that simplifies building demos or web applications without requiring any JavaScript, CSS, or web hosting expertise.

We use this to host our application and create demos.
The refined code for this can be found [here](https://huggingface.co/spaces/ashmib/gemma-gemini-eu-travels).

In [ ]:
import gradio as gr
examples = [["I'm planning a vacation to France. Can you suggest a one-week itinerary including must-visit places and "
             "local cuisines to try?", None],
            ["Recommend places that are similar to Istanbul in terms of architecture", None],
            ]

demo = gr.Interface(
    fn=generate_text,
    inputs=["text",
            gr.Dropdown(
                ["google/gemma-2b-it",], label="Models", info="Will "
                                                                                                             "add "
                                                                                                             "more "
                                                                                                             "models "
                                                                                                             "later! "
            ),
            ],
    title="🇪🇺 Euro City TravelBot 🇪🇺",
    description="Travel related queries for Europe.",
    outputs=["text"],
    examples=examples,
)
demo.launch()

## Demo
![HuggingFace Spaces Demo](https://drive.google.com/uc?id=1TXKzPJ3-KyOKIv703MFf4h3MAyYBP3RS)


https://bit.ly/eu-city-reco-app


## References and Further Reading

* https://bit.ly/llm4rec

## Workshop Feedback
* https://bit.ly/ashmib-feedback


